# Exercício 3.1

Neste exercício, são apresentadas as primeiras 7 iterações das equações pedidas pelo método de Gauss-Seidel.

In [12]:
# Bibliotecas utilizadas no exercício
import numpy as np
import pandas as pd

In [13]:
# Polinômios p(x), q(x) e r(x) utilizados no método e na montagem do sistema de equações
p = 0
q = np.power(np.pi, 2)
r = lambda x: -2*q*np.sin(np.pi*x)

# Número de iterações que serão feitas
num_iters = 7

In [14]:
# Intervalo das soluções
a = 0
b = 1

In [15]:
# Função para montagem da matriz de coeficientes do sistema 
# Foi utilizada a definição na lista, já excluindo os termos iguais a 0
def getAh(n, h):
    Ah = []
    for i in range(1, n):
        linha = []
        for j in range(1, n):
            if j == i:
                linha.append(2 + h**2*q)
            elif j == i-1 and j >= 0:
                linha.append(-1)
            elif j == i+1 and j < n:
                linha.append(-1)
            else:
                linha.append(0)
        Ah.append(linha)
    return np.array(Ah)

In [16]:
# Função para montagem do vetor de termos indepententes
# Foi utilizada a definição na lista, já excluindo os termos iguais a 0
def getVh(X_s, h):
    return np.power(h, 2)*np.array([-r(i) for i in X_s[1:-1]])

In [17]:
# Função para aplicação do método de Gauss Seidell
def GaussSeidel(n):
    # Divisão do intervalo
    h = (b-a)/n
    
    # Chute inicial    
    y0 = np.zeros(n)
    
    # Valores de x para cada divisão    
    xi = np.array([a + i*h for i in range(n+1)])
    
    # Vetor com os termos independentes    
    vh = getVh(xi, h)
    
    # Matriz dos coeficientes    
    Ah = getAh(n, h)
    
    # Matriz cujas colunas representam cada iteração pelo método
    # Inicialmente com 0s, mas será preenchida    
    xk = np.zeros((num_iters+1, n-1))
    
    for i in range(len(xk)-1):
        # Pega a iteração anterior        
        arofxs = xk[i]
        for j in range(len(xk[0])):
            # Inverso do coeficiente diagonal que multiplicará o termo           
            coeff = 1/Ah[j][j]
            
            # Vetores auxiliares para os somatórios            
            yj = vh[j]
            linhaCoefs = Ah[j]
            
            # Cálculo do método de Gauss Seidel aplicado a cada x da iteração            
            xk[i+1][j] = coeff*(yj - np.sum(linhaCoefs[j+1:]*arofxs[j+1:]) - np.sum(linhaCoefs[:j]*xk[i+1][:j]))
            
    # Transforma matriz de iterações em DataFrame para melhor visualização   
    df = pd.DataFrame(xk.T)
    return df

In [18]:
dfGaussSeigel = GaussSeidel(10)
dfGaussSeigel

,0,1,2,3,4,5,6,7
0,0.0,0.029064,0.062005,0.094239,0.124173,0.151221,0.175242,0.196314
1,0.0,0.069133,0.136781,0.199603,0.256369,0.306782,0.351006,0.389433
2,0.0,0.109033,0.208643,0.297844,0.376598,0.445389,0.504964,0.556189
3,0.0,0.141404,0.265787,0.374302,0.468260,0.549067,0.618145,0.676367
4,0.0,0.161432,0.299972,0.418406,0.519204,0.604603,0.675569,0.734119
5,0.0,0.166371,0.306413,0.424000,0.522420,0.602277,0.666933,0.719398
6,0.0,0.155365,0.283710,0.389465,0.471662,0.536389,0.587949,0.629366
7,0.0,0.129313,0.233676,0.307761,0.363747,0.407300,0.441755,0.469313
8,0.0,0.090681,0.140408,0.175708,0.202385,0.223137,0.239555,0.252686


Nessa tabela, cada coluna representa uma iteração por Gauss Seidel. A primeira, é o vetor de 0s criado para início do cálculo. A segunda e a terceira correspondem às colunas apresentadas pelo professor na lista. Como é possível observar, os valores são iguais.

# Exercício 3.2

Neste exercício é calculado o erro $E_n$ para valores de n entre 100 e 1000, variando de 100 em 100.

Para o cálculo de $Proj_h$ é utilizada a solução conhecida, $y(x) = \sin(\pi x)$.

Para o cálculo de $y_h$ foi utilizado o método da Eliminação de Gauss para a solução dos sistemas $A_h \times w = v_h$.

Algumas funções e constantes foram reutilizadas do exercício 3.2. Portanto, não estão declaradas abaixo.

In [19]:
# Função que executa o método da Eliminação de Gauss para um sistema genérico
def EliminacaoDeGauss(coeff, resultado):    
    
    # Checa se o sistema não tem mais     
    m, n = coeff.shape 
    if ( m < n ):
        print("Não há soluções únicas")
    else:
        
        # Matriz auxiliar de 0s para construção de matriz escalonada        
        l = np.zeros((n,n))
        
        # Checa se há algum 0 na diagonal da matriz de coeficientesz        
        for i in range(n):
            if (coeff[i][i] == 0):
                print("Sistema impossível")
        
        # Iteração para aplicação do método        
        for k in range(n - 1):         
            for i in range(k + 1, n):
                # Razão entre os coeficientes                
                l[i][k] = coeff[i][k] / coeff[k][k]         
                for j in range(m):
                    # Coeficiente para eliminar a linha                     
                    coeff[i][j] = coeff[i][j] - l[i][k] * coeff[k][j]
                # Mesma soma no vetor de termos independentes                
                resultado[i] = resultado[i] - l[i][k] * resultado[k]
        
        # Vetor de soluções a ser preenchido                  
        x = np.zeros(n)                                            
        x[n - 1] = resultado[n - 1] / coeff[n - 1][n - 1]  

        # Preenchimento dos resultados de trás para frente (Somando os resultados anteriores)
        for i in range(n - 2, -1, -1):            
            for j in range(i + 1, n):
                resultado[i] -= coeff[i][j] * x[j] 
            x[i] = resultado[i] / coeff[i][i]
            
        return x

In [22]:
# Método para obtenção da difernça entre as soluções encontradas e o vetor de valores max(abs(Proj_h - yh))
def get_En(n):
    # Definição da divisão dos intervalos
    h = (b-a)/n
    
    # Vetor com os valores de x nas divisões do intervalo    
    xi = np.array([a + i*h for i in range(n+1)])
    
    # Valores da função conhecida para os valores de x (1 a n-1)    
    Proj_h = np.sin(np.pi*xi[1:-1])
    
    # Vetor de resultados    
    vh = getVh(xi, h)
    
    # Matriz dos coeficientes    
    Ah = getAh(n, h)
    
    # Soluções pelo método da Eliminação de Gauss    
    yh = EliminacaoDeGauss(Ah, vh)
        
    # Vetor com os módulos da subtração termo a termo dos valores calculados na função e dos valores calculados no método    
    VetorAbs = abs(Proj_h - yh)
    
    # Pega o valor máximo do vetor de módulos
    En = max(VetorAbs)
    
    return En



In [23]:
for i in range(100,1100,100):
    print(f'E_{i} = {get_En(i)}')

E_100 = 4.112368987541082e-05
E_200 = 1.0280859061406744e-05
E_300 = 4.56926456648965e-06
E_400 = 2.5702107868674773e-06
E_500 = 1.6449350030978138e-06
E_600 = 1.142314761226615e-06
E_700 = 8.392488355379868e-07
E_800 = 6.42548916962582e-07
E_900 = 5.076927249714203e-07
E_1000 = 4.1122264748949533e-07


Como é possível observar, a diferença entre os valores exatos calculados na função e os valores obtidos pela solução do método da Eliminação de Gauss se aproximam de 0 quando n cresce. Isto é:

$\lim_{n\to\infty}E_n = 0$